# The Cypher QA Chain

In [1]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY", None)

llm = ChatGoogleGenerativeAI(
    model="gemini-pro", convert_system_message_to_human=True
)

/Users/charanahu/VS_Code/GitHub/knowledge_graph/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate

In [3]:
graph = Neo4jGraph(
    url="bolt://44.203.87.67:7687",
    username="neo4j",
    password="november-bay-fillers",
)


In [4]:
CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
Convert the user's question based on the schema.

Schema: {schema}
Question: {question}
"""

In [5]:
cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

In [6]:
cypher_chain = GraphCypherQAChain.from_llm(
    llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True
)

In [7]:
cypher_chain.run("What role did Tom Hanks play in Toy Story?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Tom Hanks"})-[r:ACTED_IN]->(m:Movie {title: "Toy Story"})
RETURN r.role
Full Context:
[{'r.role': 'Woody (voice)'}]

> Finished chain.


'Tom Hanks did not play any role in Toy Story.'

In [8]:
cypher_chain.run("What movies has Tom Hanks acted in?")




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Tom Hanks"})-[:ACTED_IN]->(m:Movie) RETURN m.title
Full Context:
[{'m.title': 'Punchline'}, {'m.title': 'Catch Me If You Can'}, {'m.title': 'Dragnet'}, {'m.title': 'Saving Mr. Banks'}, {'m.title': 'Bachelor Party'}, {'m.title': 'Volunteers'}, {'m.title': 'Man with One Red Shoe, The'}, {'m.title': 'Splash'}, {'m.title': 'Big'}, {'m.title': 'Nothing in Common'}]

> Finished chain.


'Tom Hanks has acted in movies like Punchline, Catch Me If You Can, Dragnet, Saving Mr. Banks, Bachelor Party, Volunteers, The Man with One Red Shoe, Splash, Big, and Nothing in Common.'

In [9]:
cypher_chain.run("How many movies has Tom Hanks directed?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Tom Hanks"})-[d:DIRECTED]->(m:Movie)
RETURN COUNT(DISTINCT m)
Full Context:
[{'COUNT(DISTINCT m)': 2}]

> Finished chain.


'Based on the information provided, I can tell you that Tom Hanks has directed at least 2 movies.'